In [17]:
import pandas
import numpy as np

data = pandas.read_csv("assets/Datasets/public_transport_delays.csv")

In [18]:
data

,trip_id,date,time,transport_type,route_id,origin_station,destination_station,scheduled_departure,scheduled_arrival,actual_departure_delay_min,...,wind_speed_kmh,precipitation_mm,event_type,event_attendance_est,traffic_congestion_index,holiday,peak_hour,weekday,season,delayed
0,T00000,2023-01-01,05:00:00,Tram,Route_15,Station_31,Station_6,05:02:00,05:55:00,12,...,46,13.0,NaN,500,81,0,1,6,Winter,0
1,T00001,2023-01-01,05:15:00,Metro,Route_12,Station_49,Station_32,05:16:00,05:55:00,15,...,11,11.4,NaN,0,53,0,0,6,Autumn,1
2,T00002,2023-01-01,05:30:00,Bus,Route_16,Station_29,Station_42,05:33:00,06:17:00,0,...,31,14.1,Sports,0,67,1,0,6,Autumn,0
3,T00003,2023-01-01,05:45:00,Tram,Route_19,Station_26,Station_18,05:49:00,06:08:00,15,...,41,6.4,NaN,500,84,0,0,6,Winter,1
4,T00004,2023-01-01,06:00:00,Tram,Route_8,Station_18,Station_15,06:00:00,06:35:00,-1,...,30,18.5,NaN,500,46,0,0,6,Spring,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,T01995,2023-01-21,23:45:00,Bus,Route_11,Station_46,Station_39,23:46:00,00:11:00,15,...,35,4.6,NaN,0,96,0,0,5,Winter,1
1996,T01996,2023-01-22,00:00:00,Train,Route_9,Station_44,Station_42,00:03:00,00:41:00,11,...,44,15.4,Festival,0,12,0,1,6,Winter,1
1997,T01997,2023-01-22,00:15:00,Bus,Route_12,Station_4,Station_45,00:18:00,00:35:00,1,...,32,2.7,NaN,0,24,1,0,6,Summer,1
1998,T01998,2023-01-22,00:30:00,Tram,Route_17,Station_29,Station_48,00:34:00,01:18:00,7,...,35,8.8,NaN,2000,23,0,0,6,Summer,1


In [19]:
data.columns

Index(['trip_id', 'date', 'time', 'transport_type', 'route_id',
       'origin_station', 'destination_station', 'scheduled_departure',
       'scheduled_arrival', 'actual_departure_delay_min',
       'actual_arrival_delay_min', 'weather_condition', 'temperature_C',
       'humidity_percent', 'wind_speed_kmh', 'precipitation_mm', 'event_type',
       'event_attendance_est', 'traffic_congestion_index', 'holiday',
       'peak_hour', 'weekday', 'season', 'delayed'],
      dtype='object')

In [20]:
data['scheduled_departure'] = pandas.to_datetime(data['date'] + " " + data['time'], format='%Y-%m-%d %H:%M:%S')
data['peak_hour'] = data['scheduled_departure'].dt.hour.apply(lambda hour: 1 if (hour == 8 or hour == 18) else 0)

In [21]:
def corrupt(value): 
    rng = np.random.default_rng()
    randomFloat = rng.uniform(low=0.0, high=1.0)
    if randomFloat < 0.03:
        return np.nan
    else:
        return value



data['delay_offset'] = -15
rng = np.random.default_rng()
def normalOffsets(data, column_name, category, mean, std):
    selection = data[data[column_name] == category]
    return pandas.Series(np.round(rng.normal(loc=mean, scale=std, size=selection.shape[0])), index = selection.index).reindex(data.index, fill_value = 0).apply(int)

In [22]:
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'weather_condition', 'Rain', 2, 2)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'weather_condition', 'Storm', 7, 3)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'weather_condition', 'Snow', 7, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'event_type', 'Concert', 3, 2)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'event_type', 'Parade', 4, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'event_type', 'Festival', 5, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'event_type', 'Sports', 2, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'peak_hour', 1, 3, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'transport_type', 'Bus', 3, 6)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'transport_type', 'Tram', 1, 2)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'origin_station', 'Station_9', 3, 2)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'origin_station', 'Station_10', 6, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'origin_station', 'Station_11', 4, 1)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'origin_station', 'Station_12', -5, 4)
data['delay_offset'] = data['delay_offset'] + normalOffsets(data, 'destination_station', 'Station_13', 10, 6)

In [23]:
data['humidity_percent'] = data['humidity_percent'] + normalOffsets(data, 'weather_condition', 'Snow', -5, 1)
data['temperature_C'] = data['temperature_C'] + normalOffsets(data, 'weather_condition', 'Snow', -11, 1)
data['temperature_C'] = data['temperature_C'] + normalOffsets(data, 'weather_condition', 'Storm', -6, 2)
data['temperature_C'] = data['temperature_C'] + normalOffsets(data, 'weather_condition', 'Rain', -5, 1)
data['temperature_C'] = data['temperature_C'] + normalOffsets(data, 'weather_condition', 'Cloudy', -3, 1)
data['precipitation_mm'] = data['precipitation_mm'] + normalOffsets(data, 'weather_condition', 'Snow', 10, 1)
data['precipitation_mm'] = data['precipitation_mm'] + normalOffsets(data, 'weather_condition', 'Storm', 15, 7)
data['precipitation_mm'] = data['precipitation_mm'] + normalOffsets(data, 'weather_condition', 'Rain', 9, 4)
data['wind_speed_kmh'] = data['wind_speed_kmh'] + normalOffsets(data, 'weather_condition', 'Storm', 25, 5)
data = data.apply(lambda row: rng.normal(loc = 1.0, scale = 2.0) if row['weather_condition'] == 'Clear' else row['precipitation_mm'], axis = 1)
data

0       29.000000
1       21.400000
2        1.090101
3        2.826027
4       30.500000
          ...    
1995    14.600000
1996    25.400000
1997    12.700000
1998     1.008909
1999    19.000000
Length: 2000, dtype: float64

In [ ]:
data['actual_arrival_delay_min'] = data['actual_arrival_delay_min'] + data['delay_offset']

In [ ]:
data['event_type'] = data['event_type'].fillna("No Event")

In [ ]:
data['delay_offset'].mean()

In [ ]:
                                
weatherMeasurements = [
    "temperature_C", 
    "humidity_percent",
    "wind_speed_kmh",
    "precipitation_mm",
]
for column in weatherMeasurements:
    data[column] = data[column].apply(corrupt)

In [ ]:
data.columns

In [ ]:
reducedData = data[[
    "trip_id", 
    "scheduled_departure",
    "transport_type", 
    "origin_station", 
    "destination_station", 
    "actual_departure_delay_min",
    "actual_arrival_delay_min",
    "weather_condition",
    "temperature_C",
    "humidity_percent",
    "wind_speed_kmh",
    "precipitation_mm",
    "event_type",
    "peak_hour",
]]
    

In [ ]:
reducedData[weatherMeasurements].isnull().sum()

In [ ]:
reducedData

In [ ]:
reducedData.to_csv("assets/Datasets/public_transportation_delays.csv")